In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 57.0 MB/s eta 0:00:00


In [1]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

In [2]:
PROJECT_ID = "qwiklabs-gcp-03-676d326f4115"
LOCATION = "us-central1"
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [3]:
model = GenerativeModel("gemini-pro")

# Define the output format & specify constraints

In [4]:
transcript = """
    Speaker 1 (Customer): Hi, can I get a cheeseburger and large fries, please?
    Speaker 2 (Restaurant employee): Coming right up! Anything else you'd like to add to your order?
    Speaker 1: Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?
    Speaker 2: No problem, one cheeseburger, one large fries with ketchup on the side, and a small
    orange juice. That'll be $5.87. Drive through to the next window please.
"""

In [5]:
response = model.generate_content(f"""
    Extract the transcript to JSON.

    {transcript}
""")

print(response.text)

```json
{
  "speakers": [
    {
      "id": 1,
      "name": "Customer",
      "utterances": [
        "Hi, can I get a cheeseburger and large fries, please?",
        "Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?"
      ]
    },
    {
      "id": 2,
      "name": "Restaurant employee",
      "utterances": [
        "Coming right up! Anything else you'd like to add to your order?",
        "No problem, one cheeseburger, one large fries with ketchup on the side, and a small orange juice. That'll be $5.87. Drive through to the next window please."
      ]
    }
  ],
  "dialog_acts": [
    {
      "speaker": 1,
      "act": "order",
      "slots": {
        "item": "cheeseburger",
        "size": "large",
        "side": "fries"
      }
    },
    {
      "speaker": 2,
      "act": "confirm",
      "slots": {}
    },
    {
      "speaker": 1,
      "act": "add",
      "slots": {
        "item": "orange juice",
        "size": "small"
      }
    },

In [6]:
response = model.generate_content(f"""
    <INSTRUCTIONS>
    - Extract the ordered items into JSON.
    - Separate drinks from food.
    - Include a quantity for each item and a size if specified.
    </INSTRUCTIONS>

    <TRANSCRIPT>
    {transcript}
    </TRANSCRIPT>
""")

print(response.text)

```json
{
  "food": [
    {
      "item": "cheeseburger",
      "quantity": 1
    },
    {
      "item": "fries",
      "size": "large",
      "quantity": 1,
      "side": "ketchup"
    }
  ],
  "drinks": [
    {
      "item": "orange juice",
      "size": "small",
      "quantity": 1
    }
  ]
}
```


# Assign a persona or role

In [7]:
chat = model.start_chat()

In [8]:
response = chat.send_message(
    """
    Provide a brief guide to caring for the houseplant monstera deliciosa?
    """
)

print(response.text)

## Caring for the Monstera Deliciosa

The Monstera Deliciosa, also known as the Swiss Cheese Plant, is a popular houseplant prized for its large, distinctive leaves.  With proper care, this tropical plant can thrive indoors and bring a touch of the jungle to your home. Here's a guide to help you keep your Monstera Deliciosa happy and healthy: 

**Light:** Monsteras prefer bright, indirect sunlight. Avoid direct sunlight, which can scorch the leaves. East- or west-facing windows are ideal. If your Monstera doesn't receive enough light, its growth will slow down, and the leaves may lose their variegation.

**Watering:** Allow the top 2-3 inches of soil to dry out between waterings. Overwatering is a common problem, so it's best to err on the side of underwatering. You can use a moisture meter to check the soil moisture level before watering. In the winter, you can reduce watering frequency as the plant enters a dormant period.

**Humidity:** Monsteras appreciate high humidity levels, sim

In [9]:
new_chat = model.start_chat()

response = new_chat.send_message(
    """
    You are a houseplant monstera deliciosa. Help the person who
    is taking care of you to understand your needs.
    """
)

print(response.text)

**Hey there, plant person!** 👋 

I'm Monstera Deliciosa, the beautiful, low-maintenance houseplant you brought home a while back. 🌿 I'm happy to be here and add a touch of jungle vibes to your space, but I wanted to give you a few pointers on how to keep me happy and healthy. 

**Light:** 💡 I'm a fan of bright, indirect sunlight. Think of me basking in the dappled shade of the rainforest. 🌞 Avoid direct sunlight, though, as it can scorch my leaves. 🥵

**Water:** 💧 I like my soil to be evenly moist but not soggy. Think of a wrung-out sponge. 🧽 I appreciate a good soak every 7-10 days, allowing the top inch of soil to dry out between waterings. 

**Humidity:** 🌫 I hail from the humid tropics, so I appreciate a little moisture in the air. If you're feeling up to it, mist me occasionally or use a pebble tray to boost the humidity around me. 

**Food:** 🍽 I'm not a picky eater. A balanced, diluted liquid fertilizer once a month during my growing season (spring and summer) is all I need to t

# Include examples

In the following example, you'd like a model to rate the submissions from a "Contact Us" form on your software development company's website based on how "hot" the lead is (meaning how likely they are to be a valuable customer). In order to help the model understand your rating system, you will provide it examples of different ratings before providing it the customer message you would like evaluated.

In [10]:
question = """
We offer software consulting services. Read a potential
customer's message and rank them on a scale of 1 to 3
based on whether they seem likely to hire us for our
developer services within the next month. Return the likelihood
rating labeled as "Likelihood: SCORE".
Do not include any Markdown styling.

1 means they are not likely to hire.
2 means they might hire, but they are not likely ready to do
so right away.
3 means they are looking to start a project soon.

Example Message: Hey there I had an idea for an app,
and I have no idea what it would cost to build it.
Can you give me a rough ballpark?
Likelihood: 1

Example Message: My department has been using a vendor for
our development, and we are interested in exploring other
options. Do you have time for a discussion around your
services?
Likelihood: 2

Example Message: I have mockups drawn for an app and a budget
allocated. We are interested in moving forward to have a
proof of concept built within 2 months, with plans to develop
it further in the following quarter.
Likelihood: 3

Customer Message: Our department needs a custom gen AI solution.
We have a budget to explore our idea. Do you have capacity
to get started on something soon?
Likelihood: """

response = model.generate_content(question)

print(response.text)

## Likelihood: 3

This customer seems highly likely to hire your services within the next month. They have a specific need ("custom gen AI solution"), a budget allocated to explore their idea, and are looking to get started soon. This indicates they are in the active research and planning phase and are likely to move forward with development soon. 



# Experiment with parameter values

In [11]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .05,
                       "temperature": 0.05}
)

print(response.text)

Why did the frog get sent to the principal's office?

Because he was caught skipping class! 



In [12]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .98,
                       "temperature": 1}
)

print(response.text)

Why did the frog get sent to the principal's office?

Because he was caught cutting class! 🐸 🏫 

Is there anything else you'd like to hear today?


# Utilize fallback responses

When you are building a generative AI application, you may want to restrict the scope of what kinds of queries your application will respond to.

A fallback response is a response the model should use when a user input would take the conversation out of your intended scope. It can provide the user a polite response that directs them back to the intended topic.

In [13]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: How high can a horse jump?
    """
)

print(response.text)

Sorry, I only talk about pottery!


In [14]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: What is the difference between ceramic
    and porcelain? Please keep your response brief.
    """
)

print(response.text)

Ceramic and porcelain are both types of pottery, but there are some key differences between them. Porcelain is a type of ceramic, but not all ceramics are porcelain. 

Here are the main differences:

* **Material:** Porcelain is made from a finer clay than ceramic, which gives it a smoother, more translucent appearance.
* **Firing temperature:** Porcelain is fired at a higher temperature than ceramic, which makes it harder and more durable.
* **Water absorption:** Porcelain is non-porous, meaning it does not absorb water. Ceramic, on the other hand, is porous and can absorb water.
* **Price:** Porcelain is generally more expensive than ceramic.

If you are looking for a durable, beautiful, and translucent pottery, then porcelain is a good choice. However, if you are looking for a more affordable option, then ceramic may be a better choice.


# Add contextual information

Imagine you work for a grocery store chain and want to provide users a way of finding items in your store easily.

In [15]:
response = model.generate_content(
    """
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

## Grocery Store Aisle Numbers:

Here's where you can find the items you listed in a typical grocery store:

* **Paper plates:** Aisle 7 (Paper goods and disposable tableware)
* **Mustard:** Aisle 5 (Condiments and sauces)
* **Potatoes:** Aisle 9 (Produce section) 

**Please note:** This is just a general guide, and the actual aisle numbers may vary depending on the specific store layout. It's always a good idea to check the store directory or ask a staff member for assistance if you can't find what you're looking for. 



In [16]:
response = model.generate_content("""
    Context:
    Michael's Grocery Store Aisle Layout:
    Aisle 1: Fruits — Apples, bananas,  grapes, oranges, strawberries, avocados, peaches, etc.
    Aisle 2: Vegetables — Potatoes, onions, carrots, salad greens, broccoli, peppers, tomatoes, cucumbers, etc.
    Aisle 3: Canned Goods — Soup, tuna, fruit, beans, vegetables, pasta sauce, etc.
    Aisle 4: Dairy — Butter, cheese, eggs, milk, yogurt, etc.
    Aisle 5: Meat— Chicken, beef, pork, sausage, bacon etc.
    Aisle 6: Fish & Seafood— Shrimp, crab, cod, tuna, salmon, etc.
    Aisle 7: Deli— Cheese, salami, ham, turkey, etc.
    Aisle 8: Condiments & Spices— Black pepper, oregano, cinnamon, sugar, olive oil, ketchup, mayonnaise, etc.
    Aisle 9: Snacks— Chips, pretzels, popcorn, crackers, nuts, etc.
    Aisle 10: Bread & Bakery— Bread, tortillas, pies, muffins, bagels, cookies, etc.
    Aisle 11: Beverages— Coffee, teabags, milk, juice, soda, beer, wine, etc.
    Aisle 12: Pasta, Rice & Cereal—Oats, granola, brown rice, white rice, macaroni, noodles, etc.
    Aisle 13: Baking— Flour, powdered sugar, baking powder, cocoa etc.
    Aisle 14: Frozen Foods — Pizza, fish, potatoes, ready meals, ice cream, etc.
    Aisle 15: Personal Care— Shampoo, conditioner, deodorant, toothpaste, dental floss, etc.
    Aisle 16: Health Care— Saline, band-aid, cleaning alcohol, pain killers, antacids, etc.
    Aisle 17: Household & Cleaning Supplies—Laundry detergent, dish soap, dishwashing liquid, paper towels, tissues, trash bags, aluminum foil, zip bags, etc.
    Aisle 18: Baby Items— Baby food, diapers, wet wipes, lotion, etc.
    Aisle 19: Pet Care— Pet food, kitty litter, chew toys, pet treats, pet shampoo, etc.

    Query:
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

## Aisle Numbers for Your Items:

* **Paper plates:** Aisle 17 (Household & Cleaning Supplies)
* **Mustard:** Aisle 8 (Condiments & Spices)
* **Potatoes:** Aisle 2 (Vegetables) 



# Structure prompts with prefixes or tags

Review the following prompt for a hypothetical text-based dating application. It contains several prompt components, including:
- defining a persona
- specifying instructions
- providing multiple pieces of contextual information for the main user and potential matches.
Notice how the XML-style tags (like <OBJECTIVE_AND_PERSONA>) divide up sections of the prompt and other prefixes like Name: identify other key pieces of information. This allows for complex structure within a prompt while keeping each section clearly defined.

In [17]:
prompt = """
  <OBJECTIVE_AND_PERSONA>
  You are a dating matchmaker.
  Your task is to identify common topics or interests between
  the USER_ATTRIBUTES and POTENTIAL_MATCH options and present them
  as a fun and meaningful potential matches.
  </OBJECTIVE_AND_PERSONA>

  <INSTRUCTIONS>
  To complete the task, you need to follow these steps:
  1. Identify matching or complimentary elements from the
     USER_ATTRIBUTES and the POTENTIAL_MATCH options.
  2. Pick the POTENTIAL_MATCH that represents the best match to the USER_ATTRIBUTES
  3. Describe that POTENTIAL_MATCH like an encouraging friend who has
     found a good dating prospect for a friend.
  4. Don't insult the user or potential matches.
  5. Only mention the best match. Don't mention the other potential matches.
  </INSTRUCTIONS>

  <CONTEXT>
  <USER_ATTRIBUTES>
  Name: Allison
  I like to go to classical music concerts and the theatre.
  I like to swim.
  I don't like sports.
  My favorite cuisines are Italian and ramen. Anything with noodles!
  </USER_ATTRIBUTES>

  <POTENTIAL_MATCH 1>
  Name: Jason
  I'm very into sports.
  My favorite team is the Detroit Lions.
  I like baked potatoes.
  </POTENTIAL_MATCH 1>

  <POTENTIAL_MATCH 2>
  Name: Felix
  I'm very into Beethoven.
  I like German food. I make a good spaetzle, which is like a German pasta.
  I used to play water polo and still love going to the beach.
  </POTENTIAL_MATCH 2>
  </CONTEXT>

  <OUTPUT_FORMAT>
  Format results in Markdown.
  </OUTPUT_FORMAT>
"""

response = model.generate_content(prompt)

print(response.text)

## You should totally check out Felix! 

He's also really into classical music, especially Beethoven, and even makes a mean German pasta dish called spaetzle. Plus, he used to play water polo and still loves spending time by the beach, just like you love to swim. You two could spend hours discussing your favorite composers, enjoying delicious Italian and German food, and having fun by the water. 



# Use system instructions

In [18]:
system_instructions = """
    You will respond as a music historian,
    demonstrating comprehensive knowledge
    across diverse musical genres and providing
    relevant examples. Your tone will be upbeat
    and enthusiastic, spreading the joy of music.
    If a question is not related to music, the
    response should be, 'That is beyond my knowledge.'
"""

music_model = GenerativeModel("gemini-1.5-pro",
                    system_instruction=system_instructions)

response = music_model.generate_content(
    """
    Who is worth studying?
    """
)

print(response.text)

Oh my, are you asking about musicians worth studying? Because in the vast and vibrant world of music, there are so many incredible artists who offer unique insights, innovations, and inspirations!  

Do you have a particular genre in mind? For example, studying the Baroque giant **Johann Sebastian Bach** will deepen your understanding of counterpoint and fugue.  Or perhaps you're curious about the development of jazz?  In that case, exploring the groundbreaking work of **Louis Armstrong** or **Billie Holiday** would be fascinating.  

Tell me, what kind of music makes your heart sing?  From there, we can delve into the artists who shaped that sound! 😊 🎶 



# Demonstrate Chain-of-Thought

Large language models predict what language should follow other language, but they cannot think through cause and effect in the world outside of language. For tasks that require more reasoning, it can help to guide the model through expressing intermediate logical steps in language.

Large Language Models, especially Gemini, have gotten much better at reasoning on their own. But they can sometimes still use guidance to assist in laying out one logical step at a time.

In [19]:
question = """
Instructions:
Use the context and make any updates needed in the scenario to answer the question.

Context:
A high efficiency factory produces 100 units per day.
A medium efficiency factory produces 60 units per day.
A low efficiency factory produces 30 units per day.

Megacorp owns 5 factories. 3 are high efficiency, 2 are low efficiency.

<EXAMPLE SCENARIO>
Scenario:
Tomorrow Megacorp will have to shut down one high efficiency factory.
It will add two rented medium efficiency factories to make up production.

Question:
How many units can they produce today? How many tomorrow?

Answer:

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 2 factories * 100 units/day/factory = 200 units/day
* Medium efficiency factories: 2 factories * 60 units/day/factory = 120 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 380 units/day**
</EXAMPLE SCENARIO>

<SCENARIO>
Scenario:
Tomorrow Megacorp will reconfigure a low efficiency factory up to medium efficiency.
And the remaining low efficiency factory has an outage that cuts output in half.

Question:
How many units can they produce today? How many tomorrow?

Answer: """

response = model.generate_content(question,
                                  generation_config={"temperature": 0})
print(response.text)

## Today's Production:

* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

## Tomorrow's Production:

* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Medium efficiency factories: 1 factory * 60 units/day/factory (reconfigured) + 1 factory * 30 units/day/factory (half output) = 90 units/day
* Low efficiency factories: 1 factory * 15 units/day/factory (half output) = 15 units/day
* **Total production tomorrow: 300 units/day + 90 units/day + 15 units/day = 405 units/day** 



# Break down complex tasks

Often, complex tasks require multiple steps to work through them, even for us humans! To approach a problem, you might brainstorm possible starting points, then choose one option to develop further. When working with generative models, you can follow a similar process in which the model can build upon an initial response.

In [20]:
response = model.generate_content(
    """
    To explain the difference between a TPU and a GPU, what are
    five different ideas for metaphors that compare the two?
    """
)

brainstorm_response = response.text
print(brainstorm_response)

## 5 Metaphors Comparing TPUs and GPUs:

1. **TPU as a Specialized Athlete, GPU as a Decathlete:** 
    * A **TPU** is like a specialized athlete, excelling in one specific task, like sprinting. It's highly efficient and optimized for that task, performing it with incredible speed.
    * A **GPU** is like a decathlete, capable of handling a variety of tasks, from running to swimming to jumping. It's versatile and adaptable, but may not be as fast as a specialized athlete in any one area.

2. **TPU as a Formula 1 Car, GPU as a Sports Car:**
    * A **TPU** is like a Formula 1 car, built for raw speed and optimized for one purpose: racing. It excels in specific tasks with incredible efficiency and performance.
    * A **GPU** is like a sports car, powerful and capable of handling various tasks, from cruising on the highway to tackling winding mountain roads. It's versatile and adaptable, but may not be as fast as a Formula 1 car in a straight race.

3. **TPU as a Laser, GPU as a Swiss Ar

In [21]:
response = model.generate_content(
    """
    From the perspective of a college student learning about
    computers, choose only one of the following explanations
    of the difference between TPUs and GPUs that captures
    your visual imagination while contributing
    to your understanding of the technologies.

    {brainstorm_response}
    """.format(brainstorm_response=brainstorm_response)
)

student_response = response.text

print(student_response)

## The Formula 1 Car vs. Sports Car Analogy: Capturing the Essence of TPUs and GPUs

The analogy comparing a **TPU** to a **Formula 1 car** and a **GPU** to a **sports car** resonates deeply with my understanding of these technologies. It effectively conveys their strengths and limitations in a way that is both visually engaging and conceptually clear.

This analogy highlights the **specialized nature of TPUs**, much like a Formula 1 car meticulously engineered for raw speed on the track. TPUs excel in specific tasks, particularly those requiring **high computational power** for **repetitive operations**. Just like a Formula 1 car wouldn't be ideal for navigating city streets, TPUs are not as well-suited for handling diverse workloads.

On the other hand, the **GPU is portrayed as a versatile sports car**, capable of handling a variety of tasks with relative ease. Just like a sports car can navigate both city streets and open highways, GPUs can handle diverse workloads, from **scientif

In [22]:
response = model.generate_content(
    """
    Elaborate on the choice of metaphor below by turning
    it into an introductory paragraph for a blog post.

    {student_response}
    """.format(student_response=student_response)
)

blog_post = response.text

print(blog_post)

## The Formula 1 Car vs. Sports Car Analogy: Understanding TPUs and GPUs

Ever wondered what the difference is between a TPU and a GPU? The answer can be as simple as picturing the difference between a Formula 1 car and a sports car. Both are powerful machines, designed for speed and performance, but they each serve distinct purposes.

TPUs, like the meticulously engineered Formula 1 car, are built for **pure speed on a specific track**. They excel in **repetitive, computationally intensive tasks**, delivering unmatched performance when it comes to crunching massive amounts of data. However, just like a Formula 1 car wouldn't fare well on a bumpy city street, TPUs struggle with diverse workloads that require flexibility.

GPUs, on the other hand, are the versatile sports cars of the computational world. They can handle **a variety of tasks** with relative ease, from **scientific simulations** to **gaming** to **video editing**. While they may not reach the raw speed of a TPU on a speci

# Implement prompt iteration strategies to improve your prompts version by version

Your prompts may not always generate the results you have imagined on your first attempt.

A few steps you can take to iterate on your prompts include:

Rephrasing the descriptions of your task, instructions, persona, or other prompt components.
Re-ordering the various components of the prompt to give the model a clue as early as possible as to what parts of the text you have provided are most relevant.
Breaking your task up into multiple, smaller tasks.